In [ ]:
import os

import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

from src.data.tf_data import TFDataCreator
from src.data.data_augmentation import preprocess_ct, RightAngleRotation
from src.models.models import Unet
from src.models.utils import config_gpu

%matplotlib inline

In [ ]:
config_gpu("0", 4)
task = "Task04_Hippocampus"

In [ ]:
file = h5py.File(f"../data/processed/{task}/{task}_training.hdf5", "r")

In [ ]:
image_ids = list(file.keys())
ds = tf.data.Dataset.from_tensor_slices(image_ids)

In [ ]:
ds_cut = ds.shuffle(buffer_size=32).batch(2) #.repeat().batch(2).take(130)

In [ ]:
ids = list()
for i in ds_cut:
    for b in range(i.shape[0]):
        ids.append(i[b].numpy().decode("utf-8"))

In [ ]:
missing_ids = [i for i in image_ids if i not in ids]
len(missing_ids)

In [ ]:
data_creator = TFDataCreator.get(task.split("_")[0])(file,
                                                    image_ids=["hippocampus_011"],
                                                     shuffle=True,
                                                     params_augmentation={
                                                         "rotation": True,
                                                         "random_center": False,
                                                     })
ds = data_creator.get_tf_data(data_augmentation=True)


In [ ]:
df = pd.DataFrame()
for k in file.keys():
    shape = file[k]["image"][()].shape
    df = pd.concat([df, pd.DataFrame({
        "shape_0": [shape[0]],
        "shape_1": [shape[1]],
        "shape_2": [shape[2]],
    }, index=[k])])


In [ ]:
image = file["hippocampus_011"]["image"][()]
label = file["hippocampus_011"]["label"][()]
image.shape

In [ ]:
image, label = data_creator._pad_image(image, label)
image.shape

In [ ]:
yo = ds.batch(4).repeat().as_numpy_iterator()

In [ ]:
x, y = next(yo)

In [ ]:
s = 32
b = 0
plt.subplot(1, 4, 1)
plt.imshow(x[b, :, :, s, 0])
plt.subplot(1, 4, 2)
plt.imshow(y[b, :, :, s, 0])
plt.subplot(1, 4, 3)
plt.imshow(y[b, :, :, s, 1])
plt.colorbar()
plt.subplot(1, 4, 4)
plt.imshow(y[b, :, :, s, 2])


In [ ]:
model = Unet(output_channels=2, last_activation="softmax", n_features=[12, 24, 48, 96, 192])

In [ ]:
y_pred = model(x)

In [ ]:
s = 32
b = 0
plt.subplot(1, 3, 1)
plt.imshow(x[b, :, :, s, 0])
plt.subplot(1, 3, 2)
plt.imshow(y[b, :, :, s, 0])
plt.subplot(1, 3, 3)
plt.imshow(y_pred[b, :, :, s, 0])


In [ ]:
np.unique(y)

In [ ]:
from src.models.losses import dice_loss

In [ ]:
x.shape

In [ ]:
f = lambda y_true, y_pred: dice_loss(y_true[..., 0], y_pred[..., 0])


In [ ]:
f(y, model(x)).numpy()